<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Premortem Decision Analysis  

Decision Intelligence applied in this module:  
* Using Premortem Decision Analysis to improve decision quality & outcomes  
* Using a Premortem decision output for improving of Gathering Intelligence 
* Attain different perspectives using mutliple decision frames  
* Apply a Decision Framework to Premortem Decision Analysis 


### What is a Premortem (Pre-Mortem)  

You have probably heard of a **postmortem (post-mortem)**, which is a look back at an actual failure (or success) to understand why it happened and learn from it. In the context of decision making, after the decision outcome is clear you can analyze why the result occured. 

In a **premortem (pre-mortem)** the outcome is "imagined" and then it is figured out why that potential "imagined" outcome happened. In a premortem a planning exercise where a team imagines that a project or decision has already succeeded or failed. The team then works backward, brainstorming what might have caused things to go wrong. This approach allows everyone to spot potential problems and fix them before starting the project. Therefore, it’s a bit like predicting the future and using that prediction to make the project stronger.  

### Example Premortem Scenario  

Imagine you're about to buy a new car. Let's "imagine" how the purchase results in either two scenarios a **disaster** or a **successful** decision. 

#### Disaster Decision Premortem  

In a **premortem (pre-mortem)**, you pretend the purchase turned out to be a disaster. Now, ask yourself: What went wrong?

Here are some potential reasons:  

* **Financial issues:** You realize later you can’t comfortably afford the monthly payments or maintenance costs.  
* **Unexpected repairs:** The car has hidden mechanical problems, leading to costly repairs.  
* **Insurance costs:** Insurance is much higher than expected, impacting your budget.
* **Poor fuel efficiency:** The car uses more gas than you thought, making it expensive to drive.
Resale value drop: The car’s value plummets quickly, leaving you with less if you want to sell or trade it.  

#### Successful Decision Premortem

In a premortem for a successful car purchase, imagine you bought the car, and it turned out to be a fantastic decision. Now, ask yourself: What went right?

Here are some potential reasons:

* **Affordable and realistic budget:** You set a budget that allowed for comfortable monthly payments and covered insurance, fuel, and maintenance without straining your finances.  
* **Thorough inspection:** You got a detailed inspection before buying, which confirmed the car was in excellent condition and minimized the risk of future repairs.
* **Reliable model choice:** You researched models with a reputation for reliability, so you experience fewer repairs and smooth performance.
* **Fuel efficiency matches needs:** You chose a car that meets your fuel efficiency needs, saving money on gas and fitting well with your daily driving habits.   
* **High resale value:** You picked a make and model that holds its value well, giving you confidence that it’s a good long-term investment.

Imagining these successful outcomes helps you focus on these actions—like sticking to budget, getting a full inspection, and choosing a reliable model—so you’re more likely to make a purchase you’ll be happy with.

By imagining these issues in advance, you could decide to thoroughly check maintenance records, test drive more carefully, or reassess your budget to avoid these potential problems.

How can Generative AI help? Using Generative AI, the LLM models can be used to "imagine" outcomes and also brainstorm potential reasons why that certain outcome occured. 

### 1 - Premortem Example with o1 Reasoning Models

In [7]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.25.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.25.0-alpha"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:o1Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:o1APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:o1ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.SemanticKernel, 1.25.0 Microsoft.SemanticKernel.Plugins.Core, 1.25.0-alpha

In [8]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.25.0"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages microsoft.extensions.dependencyinjection, 8.0.0 Microsoft.SemanticKernel, 1.25.0

Using Azure OpenAI Service


In [9]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";
var decisionAnalysisPrompt = """
Perform an advanced premortem analysis on the following scenario:
You are deciding to purchase a new car versus leasing a car.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: {systemPromptForDecisions}

Request from the user: {decisionAnalysisPrompt}
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(decisionAnalysisPrompt);
Console.WriteLine(simplePromptResponse.GetValue<string>());

**Advanced Premortem Analysis: Evaluating the Decision to Purchase vs. Lease a New Car**

---

**Scenario Overview:**

Imagine that three years after deciding to either purchase or lease a new car, the decision has resulted in significant negative consequences. Your financial stability, personal satisfaction, and overall well-being have been adversely affected. The goal of this premortem analysis is to identify the potential reasons for this failure, understand the risks inherent in each option, and develop strategies to mitigate these risks before making the decision.

---

### **Potential Failures and Risks Associated with Purchasing a New Car**

1. **Financial Overextension:**
   - **Underestimating Total Costs:** You failed to account for all the expenses associated with owning the car, including taxes, fees, insurance, maintenance, and repairs.
   - **High Monthly Payments:** The loan payments are higher than comfortably affordable, leading to strain on your monthly budget.
   - *

In [10]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";
var decisionAnalysisPrompt = """
Perform an advanced premortem analysis on the following scenario:
You are deciding to purchase a new car versus leasing a car.

Output the risk factors, uncertainties, and potential outcomes ONLY in a Markdown table.
Create a column for each option (Purchase, Lease) and which risk factor or uncertainty applies to each option.
Do not surround the markdown with triple backticks.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: {systemPromptForDecisions}

Request from the user: {decisionAnalysisPrompt}
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(decisionAnalysisPrompt);
Console.WriteLine(simplePromptResponse.GetValue<string>());

| Risk Factor / Uncertainty / Potential Outcome | Purchase                                                                                                | Lease                                                                                                    |
|-----------------------------------------------|-----------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------|
| High Upfront Cost                             | High initial payment required for down payment and taxes                                                  | Lower initial payment; typically only first month's payment and fees                                     |
| Depreciation of Vehicle Value                 | Owner bears the full depreciation cost over time                                                          | Depreciation is factored into lease payments; les

| Risk Factor / Uncertainty / Potential Outcome | Purchase                                                                                                | Lease                                                                                                    |
|-----------------------------------------------|-----------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------|
| High Upfront Cost                             | High initial payment required for down payment and taxes                                                  | Lower initial payment; typically only first month's payment and fees                                     |
| Depreciation of Vehicle Value                 | Owner bears the full depreciation cost over time                                                          | Depreciation is factored into lease payments; less concern for lessee                                     |
| Maintenance Costs After Warranty              | Owner responsible for all maintenance and repair costs after warranty expiration                          | Maintenance may be covered under lease terms; newer cars less likely to need repairs                      |
| Resale Value Uncertainty                      | Uncertain future resale value can affect total cost of ownership                                          | Not applicable; no need to resell the vehicle at end of lease                                             |
| Financing Costs (Interest Rates)              | Interest rates on auto loan affect total purchase cost                                                    | Interest rates affect lease payments; money factor (lease equivalent of interest rate) applies            |
| Changes in Personal Circumstances             | Financial hardship could lead to loan default or need to sell the car                                     | Financial hardship could lead to lease default with possible penalties                                    |
| Technological Obsolescence                    | Owner may be stuck with outdated technology or features                                                   | Ability to upgrade to newer model with latest technology at end of lease term                             |
| Opportunity Cost of Capital                   | Capital tied up in vehicle ownership could be used elsewhere                                              | Less capital required upfront; frees up funds for other investments                                       |
| Mileage Limits and Penalties                  | No mileage restrictions; can drive unlimited miles                                                        | Mileage limits apply; excess mileage results in additional fees                                           |
| Wear and Tear Charges                         | Owner bears cost of excessive wear but has control over repairs                                           | Excess wear and tear charges may apply at lease end                                                       |
| No Equity Built in Vehicle                    | Equity builds over time; potential asset at end of loan                                                   | No equity or ownership; payments do not contribute to ownership                                           |
| Early Termination Fees                        | Can sell the car at any time, though may incur loss if loan balance exceeds car value                     | Early lease termination can result in substantial fees and penalties                                      |
| Insurance Gap Coverage Needed                 | May be optional; depreciation could result in owing more than car's value                                 | Often required; protects against owing the difference between insurance payout and lease balance          |
| Flexibility to Modify Car                     | Full freedom to customize or modify the vehicle                                                           | Modifications typically prohibited; must return car in original condition                                 |
| Total Cost Over Time                          | Potentially lower total cost if kept long-term and maintenance costs are manageable                       | Potentially higher total cost over similar period due to continuous lease payments                        |
| Obligation Period                             | Commitment until loan is paid off or car is sold                                                          | Fixed lease term commitment; less flexibility to exit agreement early                                     |
| Impact of Market Conditions                   | Changes in used car market affect resale value and total cost of ownership                                | Lessee is insulated from market fluctuations; residual value risk is on the lessor                        |
| Tax Considerations                            | Possible tax deductions for interest paid (subject to tax laws)                                           | Lease payments may be tax-deductible if car is used for business purposes                                 |
| Credit Score Impact                           | Loan default negatively impacts credit score                                                              | Lease default negatively impacts credit score                                                             |
| End of Term Options                           | Own the car outright, sell, or trade in for a new vehicle                                                 | Options to return the car, purchase it at residual value, or lease a new vehicle                          |
| Flexibility of Use                            | Unlimited personal use; ownership allows for any usage                                                    | Use is limited by lease terms; restrictions on commercial use or ride-sharing services may apply          |